In [2]:
import psycopg2
import pandas as pd

cnn = psycopg2.connect("host=kaggle-talkingdata-adtracking-fraud-detection.civd5r1fqbzl.us-east-2.rds.amazonaws.com\
                       port=5432\
                       dbname=kaggle\
                       user=kurobeer\
                       password=pass0123")
cur = cnn.cursor()
cnn.get_backend_pid()

31307

In [3]:
cur.execute('select relname from pg_stat_user_tables;')
cur.fetchone()

('click_data',)